# Refacto events

- Add id, project_id, and org_id to EventDefinition
- Create a separate collection EventDefinition
- Resolve the Events to their EventDefinition based on their name

In [ ]:
# Set current working dir
import os
from dotenv import load_dotenv

# Path with .env
os.chdir("/Users/nicolasoulianov/phospho/phospho/backend/scripts")

load_dotenv()

In [ ]:
import pymongo

URL = os.getenv('MONGODB_URL')
DB = os.getenv('MONGODB_NAME')

# Initialize pymongo client
client = pymongo.MongoClient(URL)
db = client[DB]

DB

## Add id, project_id, org_id to EventDefinition

In [ ]:
from phospho.models import EventDefinition
from phospho.utils import generate_uuid
from copy import deepcopy

# settings.events is a Dict[str, EventDefinition]
# Change the EventDefinition to add an id, project_id and org_id 

projects = list(db["projects"].find())

for project in projects:
    event_definitions = project.get("settings", {}).get("events", {})
    old_event_definitions = deepcopy(event_definitions)
    print(f"Updating project with {len(event_definitions)} events")
    for event_name, event_definition in event_definitions.items():
        if "id" not in event_definition:
            event_definition["id"] = generate_uuid()
            print(f"Updating event {event_name} with id {event_definition['id']}")
        if "project_id" not in event_definition:
            event_definition["project_id"] = project["id"]
            print(f"Updating event {event_name} with project_id {event_definition['project_id']}")
        if "org_id" not in event_definition:
            event_definition["org_id"] = project["org_id"]
            print(f"Updating event {event_name} with org_id {event_definition['org_id']}")
    if event_definitions != old_event_definitions:
        db["projects"].update_one({"id": project["id"]}, {"$set": {"settings.events": event_definitions}})


## Create a separate collection EventDefinition

In [ ]:
# Create a separate EventDefinition collection with the same data

projects = list(db["projects"].find())

all_event_definitions = []
for project in projects:
    event_definitions = project.get("settings", {}).get("events", {})
    for event_name, event_definition in event_definitions.items():
        all_event_definitions.append(event_definition)

db["event_definitions"].insert_many(all_event_definitions)

In [ ]:
# When fetching a project, now your need to get the event definitions in a separate collection

# project = db["projects"].aggregate(
#     [
#         {"$match": {"id": "3dffe45d7c1044eaa11ac4c0e1b082c8"}},
#         {
#             "$lookup": {
#                 "from": "event_definitions",
#                 "localField": "id",
#                 "foreignField": "project_id",
#                 "as": "settings.events",
#             }
#         },
#         # The lookup operation turns the events into an array of EventDefinitions
#         # Convert into a Mapping {eventName: EventDefinition}
#         {
#             "$addFields": {
#                 "items": {
#                     "$arrayToObject": {
#                         "$map": {
#                             "input": "$settings.events",
#                             "as": "item",
#                             "in": {"k": "$$item.event_name", "v": "$$item"},
#                         }
#                     }
#                 }
#             }
#         },
#     ]
# )

# list(project)

## Resolve the Events to their EventDefinition

In [ ]:
events = list(
    db["events"].aggregate(
        [
            {
                "$match": {
                    "removed": {"$ne": True},
                }
            },
            {
                "$lookup": {
                    "from": "event_definitions",
                    "localField": "event_definition.id",
                    "foreignField": "event_definition.id",
                    "as": "event_definition",
                }
            },
            {
                "$match": {
                    "$or": [
                        {"event_definition": {"$exists": False}},
                        {"event_definition.id": {"$exists": False}},
                        {"event_definition.project_id": {"$exists": False}},
                        {"event_definition.org_id": {"$exists": False}},
                    ],
                }
            },
        ]
    )
)
events

In [ ]:
from phospho.models import Event
from tqdm import tqdm
# Each event has a .event_definition field, which is a copy of the old EventDefinition
# Update it so that now it has the latest version of the EventDefinition. Do the matching based on the event.event_name and event.project_id



for event in tqdm(events):
    event_definition_in_event = event.get("event_definition")
    if event.get("removed") and event.get("removal_reason") is None:
        db["events"].update_one(
            {"id": event["id"]},
            {
                "$set": {
                    "removed": True,
                    "removal_reason": "removed_by_user",
                },
            },
        )
    if (
        event_definition_in_event
        and "id" in event_definition_in_event
        and (
            "project_id" not in event_definition_in_event
            or "org_id" not in event_definition_in_event
        )
    ):
        print(f"Updating event with missing project_id and org_id")
        org_id = event["org_id"]
        project_id = event["project_id"]
        db["events"].update_one(
            {"id": event["id"]},
            {
                "$set": {
                    "event_definition.org_id": org_id,
                    "event_definition.project_id": project_id,
                },
            },
        )

    if not event_definition_in_event or "id" not in event_definition_in_event:
        event_definition = db["event_definitions"].find_one(
            {
                "event_name": event["event_name"],
                "project_id": event["project_id"],
                "removed": {"$ne": True},
            }
        )
        if not event_definition:
            if (
                event.get("removed")
                and event.get("removal_reason") == "removed_by_script"
            ):
                continue
            else:
                print(
                    f"Event {event['id']} has no matching event definition. Mark as removed"
                )
                db["events"].update_one(
                    {"id": event["id"]},
                    {
                        "$set": {
                            "removed": True,
                            "removal_reason": "removed_by_script",
                        },
                    },
                )
                continue
        else:
            print(f"Updating event with missing event_definition")
            db["events"].update_one(
                {"id": event["id"]}, {"$set": {"event_definition": event_definition}}
            )
    else:
        print(f"Event {event['id']} already has a proper event definition")

# Update event definitions without event names based on project.events

In [ ]:
event_definitions_without_name = list(
    db["event_definitions"].find(
        {
            "event_name": {"$exists": False},
            "removed": {"$ne": True},
        }
    )
)
len(event_definitions_without_name)

In [ ]:
event_definitions_without_name

In [ ]:
from phospho.utils import generate_uuid
from tqdm import tqdm

for event_definition in tqdm(event_definitions_without_name):
    # Fetch the event name from the project
    project = db["projects"].find_one({"id": event_definition["project_id"]})
    if event_definition.get("id") is None:
        event_definition["id"] = generate_uuid()
    # Find the event that have the same description
    matched_event = False
    for event_name, project_event_definition in project["settings"].get("events").items():
        if event_definition["description"] == project_event_definition["description"]:
            print(f"Updating event definition {event_definition['id']} with event name {event_name}")
            matched_event = True
            try:
                db["event_definitions"].update_one(
                    {
                        "project_id": project["id"],
                        "description": event_definition["description"],
                    },
                    {"$set": {"event_name": event_name, "id": event_definition["id"], "removed": False, "removal_reason": None}},
                )
            except Exception as e:
                print(e)
            break

    # No event found, mark as removed
    if not matched_event:
        print(f"Marking event definition {event_definition['id']} as removed")
        try:
            db["event_definitions"].update_one(
                {"id": event_definition["id"]},
                {"$set": {"removed": True, "removal_reason": "removed_by_script"}},
            )
        except Exception as e:
            print(e)
